## Data Preparation and Transformation

The code in this section is responsible for loading, processing, and transforming the data sets for USDT, USDC, and DAI. It includes functions for reading the transaction data, filtering out exchange-related transactions, and aggregating the data for analysis.

In [7]:
import pandas as pd
cex = pd.read_csv('./data/cex.csv')
USDC = 'USDC'
USDT = 'USDT'
DAI = 'DAI'

def load_files(stablecoin_name: str):
    # Merge datasets
    df = pd.DataFrame()
    for month in range(1, 11): #until 11 for October'23
        file_name = f'{stablecoin_name}_2023.{month:02d}-.csv'
        current_df = pd.read_csv(f'../data/2023/{stablecoin_name}/{file_name}')
        df = pd.concat([df, current_df], ignore_index=True)
    print(f'✅ {stablecoin_name} files successfully loaded.')
    return df

def transform_data(df, stablecoin_name: str, kyc: bool):
    # Create a mask for CEX 'from' addresses and exclude those rows
    from_cex_mask = df['from'].isin(cex['address'])
    df_filtered = df[~from_cex_mask]

    # Depending on the KYC flag, include or exclude 'to' CEX transactions
    if kyc:
        # For KYC data, include only transactions TO CEX Proxy addresses
        df_all = df_filtered[df_filtered['to'].isin(cex['address'])]
    else:
        # For non-KYC data, exclude transactions TO CEX Proxy addresses
        # but include txn from EOA to CEX User addresses
        df_all = df_filtered[~df_filtered['to'].isin(cex['address'])]

    # Filter for retail transactions (value between 1 and 10000)
    retail_mask = df_all['value'].between(1, 10000)
    df_retail = df_all[retail_mask]

    # Group by 'date' and perform the aggregations
    agg_df = df_all.groupby('date')['value'].agg(['sum']).reset_index()
    retail_agg_df = df_retail.groupby('date')['value'].agg(['sum']).reset_index()

    # Count unique 'from' addresses
    wallets = df_all.groupby('date')['from'].nunique().rename('all_wallets' if not kyc else 'kyc_wallets')
    retail_wallets = df_retail.groupby('date')['from'].nunique().rename('retail_wallets' if not kyc else 'kyc_retail_wallets')

    # Merge the aggregated data with the unique wallets count
    df_merged = pd.merge(wallets, agg_df, on='date', how='left')
    df_merged = pd.merge(df_merged, retail_wallets, on='date', how='left')
    df_merged = pd.merge(df_merged, retail_agg_df, on='date', how='left', suffixes=('', '_retail'))

    all_vol = 'all_vol' if not kyc else 'kyc_vol'
    retail_vol = 'retail_vol' if not kyc else 'kyc_retail_vol'

    # Rename columns
    df_merged.rename(columns={'sum': all_vol, 'sum_retail': retail_vol}, inplace=True)

    # Add asset type
    df_merged['asset'] = stablecoin_name

    # Format the date as 'YYYY-MM'
    df_merged['date'] = pd.to_datetime(df_merged['date']).dt.to_period('M')

    print(f'✅ {stablecoin_name} {'KYC' if kyc else ''} files successfully transformed.')

    return df_merged

def save_data(df, file_name: str):
    df.to_csv(f"./data/{file_name}.csv", index=False)
    print(f'✅ File `{file_name}` successfully saved.')


In [3]:
# Load raw data
df_usdt = load_files(USDT)
df_usdc = load_files(USDC)
df_dai = load_files(DAI)


✅ USDT files successfully loaded.
✅ USDC files successfully loaded.
✅ DAI files successfully loaded.


In [8]:
# Transform data
df_usdt_unique = transform_data(df_usdt, USDT, False)
df_usdc_unique = transform_data(df_usdc, USDC, False)
df_dai_unique = transform_data(df_dai, DAI, False)
df_usdt_unique_kyc = transform_data(df_usdt, USDT, True)
df_usdc_unique_kyc = transform_data(df_usdc, USDC, True)
df_dai_unique_kyc = transform_data(df_dai, DAI, True)

✅ USDT  files successfully transformed.
✅ USDC  files successfully transformed.
✅ DAI  files successfully transformed.
✅ USDT KYC files successfully transformed.
✅ USDC KYC files successfully transformed.
✅ DAI KYC files successfully transformed.


In [10]:
# Save DataFrames into csv files
save_data(df_usdt_unique, "df_usdt_unique")
save_data(df_usdc_unique, "df_usdc_unique")
save_data(df_dai_unique, "df_dai_unique")
save_data(df_usdt_unique_kyc, "df_usdt_unique_kyc")
save_data(df_usdc_unique_kyc, "df_usdc_unique_kyc")
save_data(df_dai_unique_kyc, "df_dai_unique_kyc")

✅ File `df_usdt_unique` successfully saved.
✅ File `df_usdc_unique` successfully saved.
✅ File `df_dai_unique` successfully saved.
✅ File `df_usdt_unique_kyc` successfully saved.
✅ File `df_usdc_unique_kyc` successfully saved.
✅ File `df_dai_unique_kyc` successfully saved.
